# Hoja de Trabajo #2
## Deep Learning y Sistemas Inteligentes

### Miembros
- Fernanda Esquivel 21542
- Melissa Pérez 21385

### Link al repositorio
El repositorio puede ser visualizado [acá](https://github.com/FerEsq/DL-HDT-02)
